# Hydrogen Energy and bond length

In this project we will calculate the energy of a hydrogen molecule, consisting of two positive protons with a fixed position and two electrons 'circling around'. We will use the Variational Monte Carlo method using a trial wavefunction for the electrons, with several parameters which can be tuned to minimize the Energy. This energy depends in turn on the distance $s$ between the atoms. The ultimate goal is to find the distance $s$, for which the Energy is minimized. 

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import optimize
%matplotlib inline

In [2]:
#This block is needed to create homogeneous plots throughout the project
fparams = {'axes.labelsize': 22,
           'axes.titlesize': 25,
           'axes.linewidth': 1.5,
           'font.size': 16,
           'legend.fontsize': 20,
           'font.family': 'serif',
           'font.serif': 'Computer Modern Roman',
           'xtick.labelsize': 15,
           'xtick.major.size': 5.5,
           'xtick.major.width': 1.5,
           'ytick.labelsize': 15,
           'ytick.major.size': 5.5,
           'ytick.major.width': 1.5,
           'text.usetex': True,
           'figure.autolayout': True}
plt.rcParams.update(fparams)

In [3]:
def data_blocking(datavector, l):
    """Calculate the average and standard deviation 
    of the physical input data using data_blocking,
    
    Parameters:
    -----------
    datavector: array
        The 1D - array containing the data at each timestep
    l: int
       Block length to be used with data blocking
    
    Returns:
    -----------
    tuple of floats
         Average and standard deviation of the data

    """
    
    #Divide the datavector in blocks
    blocks = [datavector[i:i+l] for i in range(0, len(datavector), l)]
    
    #Remove the last block (with probably different length)
    blocks = blocks[:-1] 
    
    mean_array = [np.mean(A) for A in blocks]
    mean_sqd = [A**2 for A in mean_array]
    mean = np.mean(mean_array)
    
    std = np.sqrt(np.mean(mean_sqd) - mean**2)
    
    return mean, std

## The Hydrogen Molecule

Now we continue to the final assignment: The Hydrogen molecule.

We use the function
$$\Psi(\vec{r}_1, \vec{r}_2) = \phi(\vec{r}_1)\phi(\vec{r}_2)\psi(\vec{r}_1, \vec{r}_2),$$ 

where 
$$ \phi(\vec{r}_1) = e^{-r_{1L}/a} + e^{-r_{1R}/a} = \phi_{1L} + \phi_{1R}$$
$$ \phi(\vec{r}_2) = e^{-r_{2L}/a} + e^{-r_{2R}/a} = \phi_{1L} + \phi_{1R}$$
$$ \psi(\vec{r_1}, \vec{r_2}) = exp\left[\frac{|\vec{r_1} - \vec{r_2}|}{\alpha(1 + \beta|\vec{r_1} - \vec{r_2}|)}\right]$$

We are going to approximate the integral:

$$\int\mathrm{d}^3 \vec{r}_1 \int\mathrm{d}^3 \vec{r}_2 \omega(\vec{r}_1, \vec{r}_2, s)\epsilon(\vec{r}_1, \vec{r}_2, s),$$ 

where 
$$\omega(\vec{r}_1, \vec{r}_2, s) = \frac{\Psi^2(\vec{r}_1, \vec{r}_2)}{\langle \Psi | \Psi \rangle}$$ is the weight and

$$\epsilon(\vec{r}_1, \vec{r}_2, s)$$ is the local energy


In [4]:
def find_a(s):
    """Calculate parameter 'a' using Coulomb cusp condition.
    
    Parameters:
    ----------- 
    s: float
       Distance between the protons
        
    Returns:
    -----------
    float
         Value of parameter a
    """
    
    #Define the function f which should equal to zero (Coulomb cusp condition)
    def f(a,s):
        return a * (1 + np.exp(-s/a)) - 1
    
    #Find the a which satisfies equation f = 0
    a = optimize.fsolve(f, 0.75, args = (s))
    
    return a

In [5]:
def set_and_accept_steps(Init_pos, rand_steps, accept_prob, beta, s, a, Add_s):
    """Given the initial position, determine the new position of the walkers
    using Metropolis method, 
    
    Parameters:
    -----------
    Init_pos: numpy array
        Array containing the position of all walkers 
    rand_steps: array_like
        Array with random step to be taken by each walker
    accept_prob: array_like
        Array containing random numbers between 0 and 1
    beta: float
        Wavefunction parameter
    s: float
       Wavefunction parameter (seperation protons)   
    a: float
       Wavefunction parameters trial Wavefunction
    Add_s: numpy array
        An array which is often needed for computations
        
           
    Returns: 
    ----------- 
    numpy array
            Updated position of the walkers 
    
    """
   
    New_pos = np.zeros(np.shape(Init_pos))
    
    #Calculate some distances, for each walker, which are needed for acceptance probability
    r1L = Init_pos[:, 0:3] + 0.5 * s * Add_s
    r1L_norm = np.linalg.norm(r1L, axis=1)
    r1R = Init_pos[:,0:3] - s * 0.5 * Add_s
    r1R_norm = np.linalg.norm(r1R, axis=1)
    r2L = Init_pos[:,3:6] + s * 0.5 * Add_s
    r2L_norm = np.linalg.norm(r2L, axis = 1)
    r2R = Init_pos[:,3:6] - s * 0.5 * Add_s
    r2R_norm = np.linalg.norm(r2R, axis=1)
    r12 = Init_pos[:,0:3] - Init_pos[:,3:6] 
    r12_norm = np.linalg.norm(r12, axis=1)
    
    #Set the random steps
    x_prime = Init_pos + rand_steps
    
    #Calculate the same distances for each walkers, after the steps
    r1L_prime = x_prime[:,0:3] + s * 0.5 * Add_s
    r1L_norm_prime = np.linalg.norm(r1L_prime, axis=1)
    r1R_prime = x_prime[:,0:3] - s * 0.5 * Add_s
    r1R_norm_prime = np.linalg.norm(r1R_prime, axis=1)
    r2L_prime = x_prime[:,3:6] + s * 0.5 * Add_s
    r2L_norm_prime = np.linalg.norm(r2L_prime, axis=1)
    r2R_prime = x_prime[:,3:6] - s * 0.5 * Add_s
    r2R_norm_prime = np.linalg.norm(r2R_prime, axis=1)
    r12_prime = x_prime[:,0:3] - x_prime[:,3:6]
    r12_norm_prime = np.linalg.norm(r12_prime, axis=1)
    
    
    #Calculate the acceptance probability for all walkers
    p1 = ( np.exp(-r1L_norm_prime / a) + np.exp(-r1R_norm_prime / a) )* \
         ( np.exp(-r2L_norm_prime / a) + np.exp(-r2R_norm_prime / a) ) * \
           np.exp(r12_norm_prime / (2 * (1 + beta * r12_norm_prime) ) )
    p2 = ( np.exp(-r1L_norm / a) + np.exp(-r1R_norm / a) )* \
         ( np.exp(-r2L_norm / a) + np.exp(-r2R_norm / a) ) * \
           np.exp( r12_norm / (2 * (1 + beta * r12_norm) ) ) 

    p = p1**2 * p2**(-2)
    
    #Determine for each walkers if the step is accepted
    accept = np.where(p > accept_prob)
    not_accept = np.where(p <= accept_prob)
    
    #Move if new position is accepted
    New_pos[accept,:] = x_prime[accept,:]
    New_pos[not_accept,:] = Init_pos[not_accept,:]
    
    return New_pos

In [6]:
def calc_local_energy(Pos_walkers, s, beta,  a, Add_s):
    """Calculate the local energy for every walker position
    
    Parameters:
    -----------
    Pos_walkers: numpy array
        Array containing the position of the walkers at certain timestep
    s: float
       Wavefunction parameter (seperation protons)
    beta: float
        Wavefunction parameter   
    a: float
       Wavefunction parameters trial Wavefunction
    Add_s: numpy array
        An array which is often needed for computations
    
    Returns: 
    -----------
    tuple of numpy arrays
        An array with the local energy at the position of all walkers 
        An array with a certain derivative term, needed in other function
    """

    #We calculate some distance vectors, which are needed for probability
    r1L_f = Pos_walkers[:,0:3] + s * 0.5 * Add_s
    r1L_norm_f = np.linalg.norm(r1L_f, axis=1)
    r1R_f = Pos_walkers[:,0:3] - s * 0.5 * Add_s
    r1R_norm_f = np.linalg.norm(r1R_f, axis=1)
    r2L_f = Pos_walkers[:,3:6] + s * 0.5 * Add_s
    r2L_norm_f = np.linalg.norm(r2L_f, axis=1)
    r2R_f = Pos_walkers[:,3:6] - s * 0.5 * Add_s
    r2R_norm_f = np.linalg.norm(r2R_f, axis=1)
    r12_f = Pos_walkers[:,0:3] - Pos_walkers[:,3:6]
    r12_norm_f = np.linalg.norm(r12_f, axis=1)

    #And some frequently used terms in the local energy.
    phi1L = np.exp(-r1L_norm_f / a)
    phi1R = np.exp(-r1R_norm_f / a)
    phi1 = phi1L + phi1R
    phi2L = np.exp(-r2L_norm_f / a)
    phi2R = np.exp(-r2R_norm_f / a)
    phi2 = phi2L + phi2R
    r1Lr12 = np.sum(r1L_f * r12_f, axis=1)/(r1L_norm_f * r12_norm_f)
    r1Rr12 = np.sum(r1R_f * r12_f, axis=1)/(r1R_norm_f * r12_norm_f)
    r2Lr12 = np.sum(r2L_f * r12_f, axis=1)/(r2L_norm_f * r12_norm_f)
    r2Rr12 = np.sum(r2R_f * r12_f, axis=1)/(r2R_norm_f * r12_norm_f)

    #Calculate local energy. Note that only one dimensional arrays are used in this expression
    EL = -a**(-2) + \
            a**(-1) * phi1**(-1) * (phi1L/r1L_norm_f + phi1R/r1R_norm_f) + \
            a**(-1) * phi2**(-1) * (phi2L/r2L_norm_f + phi2R/r2R_norm_f) + \
            - 1/r1L_norm_f - 1/r1R_norm_f - 1/r2L_norm_f - 1/r2R_norm_f + \
            1/r12_norm_f + \
            ( 2*a * (1 + beta * r12_norm_f)**2 )**-1 * \
            ( (phi1L*r1Lr12 + phi1R*r1Rr12)/phi1 - (phi2L*r2Lr12 + phi2R*r2Rr12)/phi2 )- \
            ( (4*beta + 1) * r12_norm_f + 4) / (4 * (1 + beta*r12_norm_f)**4 * r12_norm_f)
    
    #Calculate another term, needed for the damped steepest descent method
    deriv = -1 * r12_norm_f**2/((1 + beta*r12_norm_f)*2)
    
    return EL, deriv

In [7]:
def calc_energy(s, beta, N_steps, step_equi, N_walkers, block_len = 10000):
    """Calculate the expectation value, the variance of the energy of the
    system with the trial wavefunction, and expectation value of dE/dBeta. 

    Parameters:
    -----------
    s : float
        Wavefunction parameter (seperation protons)
    beta: float
        Wavefunction parameter 
    N_steps: int
        Number of steps for walkers
    step_equi: int
        Number of steps to reach equilibrium
    N_walkers: int
        Number of walkers
    block_len: int
        Block length used for data blocking
    
    Returns: 
    
    -----------
    tuple of floats
         The expectation value and the variance of the energy. Moreover
         returns expectation of dE/dBeta.
             
    """
        
        
    N = N_walkers
    Add_s = np.concatenate((np.ones([N,1]),np.zeros([N,2])),axis=1)
    
    #We calculate a
    a = find_a(s)

    x_init_3D = np.random.uniform(-1, 1, (N,6)) #Unif. Initial position Random Walker
    rand_matrix1_3D = np.random.uniform(size = (N, N_steps)) #To step or not to step 
    rand_matrix2_3D = np.random.uniform(-1, 1, size = (N, 6, N_steps))  #To give stepsize

    #Maximal stepsize
    d_max3D = 0.6
    
    #We set some counters and constants
    count = 0
    Pos_walkers = np.zeros((N, 6, N_steps+1))
    Pos_walkers[:,:,0] = x_init_3D
    
    EL_matrix = np.zeros((N, N_steps-step_equi))
    deriv_matrix1 = np.zeros((N, N_steps-step_equi))
    deriv_matrix2 = np.zeros((N, N_steps-step_equi))
    
    for i in range(N_steps):

        #We set the random steps and see whether they are accepted
        random_steps =  rand_matrix2_3D[:,:,i]*d_max3D
        Pos_walkers[:,:,i+1] = set_and_accept_steps(Pos_walkers[:,:,i], random_steps, rand_matrix1_3D[:,i], beta, s, a, Add_s)
        
        #Now calculating the local energy, after equilibrium
        if i >= step_equi:
            EL, deriv_EL = calc_local_energy(Pos_walkers[:,:,i+1], s, beta, a, Add_s)
            EL_matrix[:,i-step_equi] = EL
            deriv_matrix1[:,i-step_equi] = EL*deriv_EL
            deriv_matrix2[:,i-step_equi] = deriv_EL
            
    #Plot Energy
    #plt.plot(range(N_steps - step_equi), np.mean(EL_matrix + 1/s, axis = 0))
             
    #For each timestep we calculate average El and use datablocking
    E, varE = data_blocking(np.mean(EL_matrix,0), block_len)
    steep_descent = 2*(np.mean(deriv_matrix1) - E*np.mean(deriv_matrix2))
    
    final_E = E + 1/s   #Add proton-proton (Coulomb) interaction
    return [final_E, varE, steep_descent]

In [8]:
def minimize_beta(s, N_steps, step_equi, N_walkers, init_beta, iter = 10):
    
    """Calculate the value for the parameter beta, for which the energy of the
     system with the trial wavefunction is minimized.

    Parameters:
    -----------
    s : float
        Wavefunction parameter (seperation protons)       
    N_steps: int
        Number of steps for walkers
    step_equi: int 
        Number of steps to reach equilibrium
    N_walkers: int
        Number of walkers
    block_len: int
        block length used for data blocking
    
    Returns: 
    -----------
    tuple of floats
        The value of beta for which the energy is minimized 
        the expectation value of this energy.
        dE/dBeta for this minimal beta. 
    """
    
    #We set some counters and constants
    beta_low = init_beta
    E_low = 1000
    beta = init_beta
    countsteep = 0

    while countsteep < iter:

        E, Var, steep_descent  = calc_energy(s, beta, N_steps, step_equi, N_walkers, block_len = 200)
        countsteep += 1
        
        #Update beta
        beta = beta - 5*steep_descent
    
    return beta, E, steep_descent

## Determine optimal Beta 

### Initialize parameters and constants 

In [ ]:
N_steps = 5000
step_equi = 1000
Nwalkers = 400
Init_beta = 0.7

Find optimal values of $\beta$

In [ ]:
#The values for s, seperation between two protons
s_vec = np.linspace(1,1.8,40)


#Array to store optimal beta values (and derivate dE/dBeta, to check results)
beta_min = np.zeros(len(s_vec))
deriv = np.zeros(len(s_vec))

#for each seperation s, we find the optimal beta and we save the data
for i,s in enumerate(s_vec):
    beta_min[i], Energy, deriv[i] = minimize_beta(s, N_steps, step_equi, Nwalkers, 
                                                          Init_beta, iter = 10)

    print(i, end = " ")

np.save('.\Data\VMC\min_beta', beta_min)
np.save('.\Data\VMC\s_vec', s_vec)
np.save('.\Data\VMC\deriv', deriv)

Now we run VMC one more time for the optimal $\beta$, using data blocking

In [ ]:
N_steps = 81000
step_equi = 1000
Nwalkers = 400

In [ ]:
s_vec = np.load('.\Data\VMC\s_vec.npy')
beta_optim = np.load('.\Data\VMC\min_beta.npy')
E_VarE_array = np.zeros((2,len(s_vec)))

#for each seperation s, we calculate the energy (and the uncertainty)
for i,s in enumerate(s_vec):
    beta = beta_optim[i]
    E_VarE_array[:,i] = calc_energy(s, beta, N_steps, step_equi, Nwalkers, 
                                        block_len = 400)[0:2]
    print(i, end = " ")

    np.save('.\Data\VMC\E_final2', E_VarE_array[0,:])
    np.save('.\Data\VMC\Var_final2', E_VarE_array[1,:])

### Plot results

In [ ]:
E_final = np.load('.\Data\VMC\E_final.npy')
Var_final = np.load('.\Data\VMC\Var_final.npy')
s_vector = np.linspace(1,1.8, 40)

plt.errorbar(s_vector, E_final, Var_final)
plt.title('Energy of Hydrogen molecule', fontweight = 'bold')
plt.xlabel(r'seperation protons ($a_0$)')
plt.ylabel('Energy (a.u)')
plt.savefig(".\Verslag\img\VMC_final_improv.pdf")
